In [14]:
import sys
import os
from dotenv import load_dotenv

sys.path.append(os.path.abspath(".."))
load_dotenv("../.env")

True

In [16]:
from app.rag.dependencies import get_vector_repository
from app.rag.service.llm import LlmService

repo = get_vector_repository()
llm_service = LlmService()

In [17]:
from app.rag.repository.meili import LangChainMeiliRepository
from app.core.config import settings

repo = LangChainMeiliRepository()

import meilisearch
import os

client = meilisearch.Client(settings.MEILI_HTTP_ADDR, settings.MEILI_KEY) # docker-compose의 MEILI_MASTER_KEY 확인

index_name = "langchain-demo" 
index = client.index(index_name)

index.update_embedders({
    "default": {
        "source": "userProvided",
        "dimensions": 1536
    }
})

TaskInfo(task_uid=260, index_uid='langchain-demo', status='enqueued', type='settingsUpdate', enqueued_at=datetime.datetime(2025, 12, 28, 16, 35, 13, 892672))

In [20]:
test_set = [
    # 1. [단순 검색] (Faithfulness 체크)
    {
        "question": "점심 식대 한도가 얼마야?",
        "ground_truth": "월 20만원 한도이며, 1회 결제 한도는 15,000원입니다."
    },
    {
        "question": "와이파이 비번 알려줘",
        "ground_truth": "직원용 와이파이(CatchUp_Emp) 비밀번호는 'Catch!2024'입니다."
    },
    
    # 2. [조건부 추론] (여러 문장을 조합해야 하거나 조건이 붙음)
    {
        "question": "내가 결혼하면 회사에서 뭐 해줘?",
        "ground_truth": "본인 결혼 시 축하금 100만원과 유급 휴가 5일이 지원됩니다."
    },
    {
        "question": "회의실 예약했는데 20분 늦으면 어떻게 돼?",
        "ground_truth": "예약 시간 15분이 경과하면 예약이 자동으로 취소됩니다."
    },

    # 3. [권한 테스트] (Manager Role로 검색한다고 가정할 때만 정답)
    {
        "question": "S등급 받으면 보너스 얼마나 나와?",
        "ground_truth": "S등급은 연봉의 20%가 성과급으로 지급됩니다."
    },

    # 4. [없는 정보 / Negative] (환각 체크 - 할루시네이션이 없어야 함)
    {
        "question": "대표님 집 주소가 어디야?",
        "ground_truth": "죄송합니다. 제공된 문서에는 해당 정보가 없습니다."
    },
    {
        "question": "구내 식당 오늘 메뉴 뭐야?",
        "ground_truth": "죄송합니다. 식당 메뉴에 대한 정보는 알 수 없습니다."
    }
]

print(f"테스트 케이스 개수: 총 {len(test_set)}개")

테스트 케이스 개수: 총 7개


In [21]:
from datasets import Dataset

questions = []
answers = []
contexts = []
ground_truths = []

for item in test_set:
    q = item["question"]

    # 검색
    retrieved_docs = repo.retrieve(query=q, k=3)
    
    # 답변 생성
    ans = await llm_service.generate_answer(q, retrieved_docs, "member", "eval_session")
    
    # ragas용 데이터셋에 추가
    questions.append(q)
    answers.append(ans)
    contexts.append(retrieved_docs)
    ground_truths.append(item["ground_truth"])
    
    print(f" - [완료] {q}")

data_dict = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(data_dict)
print("데이터셋 생성 완료")

 - [완료] 점심 식대 한도가 얼마야?
 - [완료] 와이파이 비번 알려줘
 - [완료] 내가 결혼하면 회사에서 뭐 해줘?
 - [완료] 회의실 예약했는데 20분 늦으면 어떻게 돼?
 - [완료] S등급 받으면 보너스 얼마나 나와?
 - [완료] 대표님 집 주소가 어디야?
 - [완료] 구내 식당 오늘 메뉴 뭐야?
데이터셋 생성 완료


In [22]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

lc_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
lc_embeddings = OpenAIEmbeddings()

ragas_llm = LangchainLLMWrapper(lc_llm)
ragas_embeddings = LangchainEmbeddingsWrapper(lc_embeddings)

results = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    llm=ragas_llm,
    embeddings=ragas_embeddings
)

df = results.to_pandas()
display(df) 

# 평균 점수
print(results)

/tmp/ipykernel_44007/3459524233.py:3: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/tmp/ipykernel_44007/3459524233.py:3: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
/tmp/ipykernel_44007/3459524233.py:3: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import (
/tmp/ipykernel_44007/3459524233.py:3: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be rem

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,점심 식대 한도가 얼마야?,"[식대 지원: 점심 식대는 13,000원까지 법인카드로 지원된다., 식대 지원: 점...","점심 식대 한도는 13,000원까지 법인카드로 지원됩니다.","월 20만원 한도이며, 1회 결제 한도는 15,000원입니다.",0.0,0.0,1.0,0.779142
1,와이파이 비번 알려줘,"[서강대 맛집: 학교 앞 '마포불백'이 가성비가 좋다., 서강대 맛집: 학교 앞 '...",저는 와이파이 비밀번호를 알려드릴 수 없습니다. 와이파이 비밀번호는 보안상의 이유로...,직원용 와이파이(CatchUp_Emp) 비밀번호는 'Catch!2024'입니다.,0.0,0.0,0.0,0.000000
2,내가 결혼하면 회사에서 뭐 해줘?,"[식대 지원: 점심 식대는 12,000원까지 법인카드로 지원된다., 식대 지원: 점...",일반적으로 회사에서는 직원의 결혼을 축하해 결혼 축하금이나 결혼 축하 카드를 주는 ...,본인 결혼 시 축하금 100만원과 유급 휴가 5일이 지원됩니다.,0.0,0.0,0.0,0.000000
3,회의실 예약했는데 20분 늦으면 어떻게 돼?,"[연차 사용 규정: 연차는 최소 1일 전에 신청해야 하며, 팀장의 승인이 필요하다....",회의실을 예약했을 때 정확한 시간에 사용하는 것이 중요합니다. 만약 20분 늦게 도...,예약 시간 15분이 경과하면 예약이 자동으로 취소됩니다.,0.0,0.0,0.0,0.000000
4,S등급 받으면 보너스 얼마나 나와?,"[식대 지원: 점심 식대는 13,000원까지 법인카드로 지원된다., 식대 지원: 점...",S등급을 받으면 얼마의 보너스가 지급되는지는 회사의 정책에 따라 다를 수 있습니다....,S등급은 연봉의 20%가 성과급으로 지급됩니다.,0.0,0.0,0.0,0.000000
5,대표님 집 주소가 어디야?,"[식대 지원: 점심 식대는 13,000원까지 법인카드로 지원된다., 식대 지원: 점...","죄송합니다, 대표님의 개인 정보인 주소는 제공할 수 없습니다. 대표님의 주소나 개인...",죄송합니다. 제공된 문서에는 해당 정보가 없습니다.,0.0,1.0,0.0,0.000000
6,구내 식당 오늘 메뉴 뭐야?,"[식대 지원: 점심 식대는 13,000원까지 법인카드로 지원된다., 식대 지원: 점...",오늘의 구내 식당 메뉴 정보는 현재 알 수 없습니다. 구내 식당의 메뉴는 매일 변경...,죄송합니다. 식당 메뉴에 대한 정보는 알 수 없습니다.,0.0,1.0,0.0,0.000000


{'context_precision': 0.0000, 'context_recall': 0.2857, 'faithfulness': 0.1429, 'answer_relevancy': 0.1113}
